# Training and Deploying TensorFlow Models at Scale

### Import Libraries

In [1]:
# # Python ≥3.5 is required
# import sys
# assert sys.version_info >= (3, 5)

# # # Is this notebook running on Colab or Kaggle?
# # IS_COLAB = "google.colab" in sys.modules
# # IS_KAGGLE = "kaggle_secrets" in sys.modules

# # if IS_COLAB or IS_KAGGLE:
# #     !echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" > /etc/apt/sources.list.d/tensorflow-serving.list
# #     !curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
# #     !apt update && apt-get install -y tensorflow-model-server
# #     %pip install -q -U tensorflow-serving-api

# # Scikit-Learn ≥0.20 is required
# import sklearn
# assert sklearn.__version__ >= "0.20"

# # TensorFlow ≥2.0 is required
# import tensorflow as tf
# from tensorflow import keras
# # assert tf.__version__ >= "2.0"

# # if not tf.config.list_physical_devices('GPU'):
# #     print("No GPU was detected. CNNs can be very slow without a GPU.")
# #     if IS_COLAB:
# #         print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
# #     if IS_KAGGLE:
# #         print("Go to Settings > Accelerator and select GPU.")

# # Common imports
# import numpy as np
# import os

# # to make this notebook's output stable across runs
# np.random.seed(42)
# tf.random.set_seed(42)

# # To plot pretty figures
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)

## Deploying TensorFlow models to TensorFlow Serving (TFS) 

In [2]:
# (X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
# X_train_full = X_train_full[..., np.newaxis].astype(np.float32) / 255.
# X_test = X_test[..., np.newaxis].astype(np.float32) / 255.
# X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
# y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
# X_new = X_test[:3]

In [3]:
# np.random.seed(42)
# tf.random.set_seed(42)

# model = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=[28, 28, 1]),
#     keras.layers.Dense(100, activation="relu"),
#     keras.layers.Dense(10, activation="softmax")
# ])
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer=keras.optimizers.SGD(learning_rate=1e-2),
#               metrics=["accuracy"])
# model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [4]:
# np.round(model.predict(X_new), 2)

In [5]:
# model_version = "0001"
# model_name = "my_mnist_model"
# model_path = os.path.join(model_name, model_version)
# model_path

In [6]:
# import shutil

# shutil.rmtree(model_name)

In [7]:
# tf.saved_model.save(model, model_path)

In [8]:
# for root, dirs, files in os.walk(model_name):
#     indent = '    ' * root.count(os.sep)
#     print('{}{}/'.format(indent, os.path.basename(root)))
#     for filename in files:
#         print('{}{}'.format(indent + '    ', filename))

In [9]:
# !saved_model_cli show --dir {model_path}

In [10]:
# !saved_model_cli show --dir {model_path} --tag_set serve

In [11]:
# !saved_model_cli show --dir {model_path} --tag_set serve \
#                       --signature_def serving_default

In [12]:
# !saved_model_cli show --dir {model_path} --all

In [13]:
# np.save("my_mnist_tests.npy", X_new)

In [14]:
# input_name = model.input_names[0]
# input_name

In [15]:
# !saved_model_cli run --dir {model_path} --tag_set serve \
#                      --signature_def serving_default    \
#                      --inputs {input_name}=my_mnist_tests.npy

In [16]:
# np.round([[1.1347984e-04, 1.5187356e-07, 9.7032893e-04, 2.7640699e-03, 3.7826971e-06,
#            7.6876910e-05, 3.9140293e-08, 9.9559116e-01, 5.3502394e-05, 4.2665208e-04],
#           [8.2443521e-04, 3.5493889e-05, 9.8826385e-01, 7.0466995e-03, 1.2957400e-07,
#            2.3389691e-04, 2.5639210e-03, 9.5886099e-10, 1.0314899e-03, 8.7952529e-08],
#           [4.4693781e-05, 9.7028232e-01, 9.0526715e-03, 2.2641101e-03, 4.8766597e-04,
#            2.8800720e-03, 2.2714981e-03, 8.3753867e-03, 4.0439744e-03, 2.9759688e-04]], 2)

## TensorFlow Serving

In [17]:
# os.environ["MODEL_DIR"] = os.path.split(os.path.abspath(model_path))[0]

In [18]:
# %%bash --bg
# nohup tensorflow_model_server \
#      --rest_api_port=8501 \
#      --model_name=my_mnist_model \
#      --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [19]:
# !tail server.log

In [20]:
# import json

# input_data_json = json.dumps({
#     "signature_name": "serving_default",
#     "instances": X_new.tolist(),
# })

In [21]:
# repr(input_data_json)[:1500] + "..."

In [22]:
# import requests

# SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
# response = requests.post(SERVER_URL, data=input_data_json)
# response.raise_for_status() # raise an exception in case of error
# response = response.json()

In [23]:
# response.keys()

In [24]:
# y_proba = np.array(response["predictions"])
# y_proba.round(2)

### Using the gRPC API

In [25]:
# from tensorflow_serving.apis.predict_pb2 import PredictRequest

# request = PredictRequest()
# request.model_spec.name = model_name
# request.model_spec.signature_name = "serving_default"
# input_name = model.input_names[0]
# request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [ ]:
# import grpc
# from tensorflow_serving.apis import prediction_service_pb2_grpc

# channel = grpc.insecure_channel('localhost:8500')
# predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
# response = predict_service.Predict(request, timeout=10.0)

In [26]:
# response

In [27]:
# output_name = model.output_names[0]
# outputs_proto = response.outputs[output_name]
# y_proba = tf.make_ndarray(outputs_proto)
# y_proba.round(2)

In [ ]:
output_name = model.output_names[0]
outputs_proto = response.outputs[output_name]
shape = [dim.size for dim in outputs_proto.tensor_shape.dim]
y_proba = np.array(outputs_proto.float_val).reshape(shape)
y_proba.round(2)

## Deploying a new model version

In [28]:
# np.random.seed(42)
# tf.random.set_seed(42)

# model = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=[28, 28, 1]),
#     keras.layers.Dense(50, activation="relu"),
#     keras.layers.Dense(50, activation="relu"),
#     keras.layers.Dense(10, activation="softmax")
# ])
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer=keras.optimizers.SGD(learning_rate=1e-2),
#               metrics=["accuracy"])
# history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [29]:
# model_version = "0002"
# model_name = "my_mnist_model"
# model_path = os.path.join(model_name, model_version)
# model_path

In [30]:
# tf.saved_model.save(model, model_path)

In [31]:
# for root, dirs, files in os.walk(model_name):
#     indent = '    ' * root.count(os.sep)
#     print('{}{}/'.format(indent, os.path.basename(root)))
#     for filename in files:
#         print('{}{}'.format(indent + '    ', filename))

In [32]:
# import requests

# SERVER_URL = 'http://localhost:8501/v1/models/my_mnist_model:predict'
            
# response = requests.post(SERVER_URL, data=input_data_json)
# response.raise_for_status()
# response = response.json()

In [33]:
# response.keys()

In [34]:
# y_proba = np.array(response["predictions"])
# y_proba.round(2)

## Deploy the model to Google Cloud AI Platform

In [35]:
# project_id = "onyx-smoke-242003"

In [36]:
# import googleapiclient.discovery

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_private_key.json"
# model_id = "my_mnist_model"
# model_path = "projects/{}/models/{}".format(project_id, model_id)
# model_path += "/versions/v0001/" # if you want to run a specific version
# ml_resource = googleapiclient.discovery.build("ml", "v1").projects()

In [37]:
# def predict(X):
#     input_data_json = {"signature_name": "serving_default",
#                        "instances": X.tolist()}
#     request = ml_resource.predict(name=model_path, body=input_data_json)
#     response = request.execute()
#     if "error" in response:
#         raise RuntimeError(response["error"])
#     return np.array([pred[output_name] for pred in response["predictions"]])

In [38]:
# Y_probas = predict(X_new)
# np.round(Y_probas, 2)

## Using GPUs

In [39]:
# #tf.test.is_gpu_available() # deprecated
# tf.config.list_physical_devices('GPU')

In [40]:
# tf.test.gpu_device_name()

In [41]:
# tf.test.is_built_with_cuda()

In [42]:
# from tensorflow.python.client.device_lib import list_local_devices

# devices = list_local_devices()
# devices

## Distributed Training

In [43]:
# keras.backend.clear_session()
# tf.random.set_seed(42)
# np.random.seed(42)

In [44]:
# def create_model():
#     return keras.models.Sequential([
#         keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
#                             padding="same", input_shape=[28, 28, 1]),
#         keras.layers.MaxPooling2D(pool_size=2),
#         keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
#                             padding="same"), 
#         keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
#                             padding="same"),
#         keras.layers.MaxPooling2D(pool_size=2),
#         keras.layers.Flatten(),
#         keras.layers.Dense(units=64, activation='relu'),
#         keras.layers.Dropout(0.5),
#         keras.layers.Dense(units=10, activation='softmax'),
#     ])

In [45]:
# batch_size = 100
# model = create_model()
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer=keras.optimizers.SGD(learning_rate=1e-2),
#               metrics=["accuracy"])
# model.fit(X_train, y_train, epochs=10,
#           validation_data=(X_valid, y_valid), batch_size=batch_size)

In [46]:
# keras.backend.clear_session()
# tf.random.set_seed(42)
# np.random.seed(42)

# distribution = tf.distribute.MirroredStrategy()

# # Change the default all-reduce algorithm:
# #distribution = tf.distribute.MirroredStrategy(
# #    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

# # Specify the list of GPUs to use:
# #distribution = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# # Use the central storage strategy instead:
# #distribution = tf.distribute.experimental.CentralStorageStrategy()

# #if IS_COLAB and "COLAB_TPU_ADDR" in os.environ:
# #  tpu_address = "grpc://" + os.environ["COLAB_TPU_ADDR"]
# #else:
# #  tpu_address = ""
# #resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
# #tf.config.experimental_connect_to_cluster(resolver)
# #tf.tpu.experimental.initialize_tpu_system(resolver)
# #distribution = tf.distribute.experimental.TPUStrategy(resolver)

# with distribution.scope():
#     model = create_model()
#     model.compile(loss="sparse_categorical_crossentropy",
#                   optimizer=keras.optimizers.SGD(learning_rate=1e-2),
#                   metrics=["accuracy"])

In [47]:
# batch_size = 100 # must be divisible by the number of workers
# model.fit(X_train, y_train, epochs=10,
#           validation_data=(X_valid, y_valid), batch_size=batch_size)

In [48]:
# model.predict(X_new)

In [49]:
# keras.backend.clear_session()
# tf.random.set_seed(42)
# np.random.seed(42)

# K = keras.backend

# distribution = tf.distribute.MirroredStrategy()

# with distribution.scope():
#     model = create_model()
#     optimizer = keras.optimizers.SGD()

# with distribution.scope():
#     dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().batch(batch_size)
#     input_iterator = distribution.make_dataset_iterator(dataset)
    
# @tf.function
# def train_step():
#     def step_fn(inputs):
#         X, y = inputs
#         with tf.GradientTape() as tape:
#             Y_proba = model(X)
#             loss = K.sum(keras.losses.sparse_categorical_crossentropy(y, Y_proba)) / batch_size

#         grads = tape.gradient(loss, model.trainable_variables)
#         optimizer.apply_gradients(zip(grads, model.trainable_variables))
#         return loss

#     per_replica_losses = distribution.experimental_run(step_fn, input_iterator)
#     mean_loss = distribution.reduce(tf.distribute.ReduceOp.SUM,
#                                     per_replica_losses, axis=None)
#     return mean_loss

# n_epochs = 10
# with distribution.scope():
#     input_iterator.initialize()
#     for epoch in range(n_epochs):
#         print("Epoch {}/{}".format(epoch + 1, n_epochs))
#         for iteration in range(len(X_train) // batch_size):
#             print("\rLoss: {:.3f}".format(train_step().numpy()), end="")
#         print()

## Training across multiple servers

In [50]:
# cluster_spec = {
#     "worker": [
#         "machine-a.example.com:2222",  # /job:worker/task:0
#         "machine-b.example.com:2222"   # /job:worker/task:1
#     ],
#     "ps": ["machine-c.example.com:2222"] # /job:ps/task:0
# }

In [51]:
# import os
# import json

# os.environ["TF_CONFIG"] = json.dumps({
#     "cluster": cluster_spec,
#     "task": {"type": "worker", "index": 1}
# })
# os.environ["TF_CONFIG"]

In [52]:
# import tensorflow as tf

# resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
# resolver.cluster_spec()

In [53]:
# resolver.task_type

In [54]:
# resolver.task_id

In [55]:
# %%writefile my_mnist_multiworker_task.py

# import os
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# import time

# # At the beginning of the program
# distribution = tf.distribute.MultiWorkerMirroredStrategy()

# resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
# print("Starting task {}{}".format(resolver.task_type, resolver.task_id))

# # Only worker #0 will write checkpoints and log to TensorBoard
# if resolver.task_id == 0:
#     root_logdir = os.path.join(os.curdir, "my_mnist_multiworker_logs")
#     run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
#     run_dir = os.path.join(root_logdir, run_id)
#     callbacks = [
#         keras.callbacks.TensorBoard(run_dir),
#         keras.callbacks.ModelCheckpoint("my_mnist_multiworker_model.h5",
#                                         save_best_only=True),
#     ]
# else:
#     callbacks = []

# # Load and prepare the MNIST dataset
# (X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
# X_train_full = X_train_full[..., np.newaxis] / 255.
# X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
# y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# with distribution.scope():
#     model = keras.models.Sequential([
#         keras.layers.Conv2D(filters=64, kernel_size=7, activation="relu",
#                             padding="same", input_shape=[28, 28, 1]),
#         keras.layers.MaxPooling2D(pool_size=2),
#         keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
#                             padding="same"), 
#         keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu",
#                             padding="same"),
#         keras.layers.MaxPooling2D(pool_size=2),
#         keras.layers.Flatten(),
#         keras.layers.Dense(units=64, activation='relu'),
#         keras.layers.Dropout(0.5),
#         keras.layers.Dense(units=10, activation='softmax'),
#     ])
#     model.compile(loss="sparse_categorical_crossentropy",
#                   optimizer=keras.optimizers.SGD(learning_rate=1e-2),
#                   metrics=["accuracy"])

# model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
#           epochs=10, callbacks=callbacks)

In [56]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [57]:
# import subprocess

# cluster_spec = {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}

# for index, worker_address in enumerate(cluster_spec["worker"]):
#     os.environ["TF_CONFIG"] = json.dumps({
#         "cluster": cluster_spec,
#         "task": {"type": "worker", "index": index}
#     })
#     subprocess.Popen("python my_mnist_multiworker_task.py", shell=True)

In [58]:
# %load_ext tensorboard
# %tensorboard --logdir=./my_mnist_multiworker_logs --port=6006

In [59]:
# from tensorflow import keras

# model = keras.models.load_model("my_mnist_multiworker_model.h5")
# Y_pred = model.predict(X_new)
# np.argmax(Y_pred, axis=-1)